In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.options import Options
import os
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from selenium.webdriver.common.keys import Keys
import time
from multiprocessing import Pool
start = 1676
end = 1810

rootdir = "C:/Users/minif/OneDrive/Desktop/"
ticker_list = open("QuantConnect Option Tickers.txt").read().splitlines()
ticker_list = ticker_list[start:end]
ticker_dic = {key: None for key in ticker_list}
delay = 10 # seconds
browser = webdriver.Firefox()
'''
Step 1:
    Get the dates of every ticker
Step 2:
    Get the picture for every URL
Step 3:
    Analyze the pictures for data and convert that to text
Step 4:
    Cleanup the files to make space.

Another thing, I need to download the images by using the image URLs and opening up in a new browser or tab
and then downloading it from there.  This is going to be an arduous process, but it is possible and that is all
I need to know.  First off, I will try accessing the page and making sure that my code to go to all URLs is good.
Then, I will try clicking the next button to see if I can get the next images.
Then, I will try getting the images using the above method.
Then, I will put all the images in their respective folders that I created with each ticker.
This will be a lot of data, but it will be worth it.

GOAL: Pull their entire options data library.
'''
print(len(ticker_list))

#This block pulls all the urls to check per ticker
def pull_urls_page(url_list_page):
    try:
        WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//a[contains(text(), ".zip")]')))
    except TimeoutException:
        WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//a[contains(text(), ".zip")]')))

    elements = browser.find_elements_by_xpath('//a[contains(text(), ".zip")]')
    for n in elements:
        url_list_page.append(n.text)
    
    try:
        WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="paginate_button next"]//a')))
        nextBtn = browser.find_element_by_xpath('//li[@class="paginate_button next"]//a')
        
        nextBtn.click()
        pull_urls_page(url_list_page)
    except TimeoutException:
        pass

def pull_urls(ticker):
    # date and expiration_date should be in form of YYYYMMDD
    # ticker should be lower case
    # price should be the strike price times 10,000
    
    url_list = []
    
    browser.get('https://www.quantconnect.com/data/tree/option/usa/minute/' + ticker)
    WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//option[@value="50"]')))
    
    btnFifty = browser.find_element_by_xpath('//option[@value="50"]')
    WebDriverWait(browser, delay).until(EC.element_to_be_clickable((By.XPATH, '//option[@value="50"]')))
    btnFifty.click()
    pull_urls_page(url_list)
    ticker_dic[ticker] = url_list

for t in ticker_list:
    pull_urls(t)

134


In [5]:
def write_to_csv(ticker_dic):
    ticker_name_lst = []
    ticker_val_lst = []
    
    ticker_list = list(ticker_dic.keys())

    for i in range(len(ticker_dic)):
        ticker_name_lst.append(ticker_list[i])
        ticker_val_lst.append(ticker_dic[ticker_list[i]])
        
    df = pd.DataFrame()
    df["ticker"] = ticker_name_lst
    df["urls"] = ticker_val_lst
    
    writer = pd.ExcelWriter('C:/Users/minif/OneDrive/Desktop/options_data_' + str(start) + '_' + str(end) + '.xlsx')
    df.to_excel(writer,'Sheet1')
    writer.save()
    
    return df

for key in list(ticker_dic.keys()):
    if ticker_dic[key] == None:
        del ticker_dic[key]
write_to_csv(ticker_dic)

,ticker,urls
0,date,"[20110602_quote_american.zip, 20110603_openint..."
1,date1,"[20130603_quote_american.zip, 20130604_openint..."
2,dave,"[20100426_quote_american.zip, 20100426_trade_a..."
3,db,"[20070103_openinterest_american.zip, 20070103_..."
4,db1,"[20100922_quote_american.zip, 20100922_trade_a..."
5,dba,"[20070509_openinterest_american.zip, 20070509_..."
6,dbb,"[20070509_openinterest_american.zip, 20070509_..."
7,dbc,"[20070509_openinterest_american.zip, 20070509_..."
8,dbd,"[20070103_openinterest_american.zip, 20070103_..."
9,dbe,"[20070509_openinterest_american.zip, 20070509_..."
